In [2]:
import os
import datetime
from fredapi import Fred
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd

load_dotenv(".env")
FRED_API_KEY = os.getenv("API_KEY")
fred = Fred(api_key=FRED_API_KEY)
client = MongoClient('localhost', 27017)
db = client.economic_data
collection = db.economic_collection

In [3]:
# Define your series dictionary, start and end dates as before
series_dict = {
    'bonds2tr': ('DGS2', 'M'),
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bond10tr': ('DGS10', 'M'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'cp': ('CP', 'Q')
}

# Define dates
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Initialize DataFrames
dfs = {'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}

# Function to fetch and merge series data
def fetch_and_merge(df, var_name, series_id, start, end):
    series_data = fred.get_series(series_id)
    series_data = series_data.loc[start_date:end_date]
    series_df = series_data.to_frame(name=var_name)
    series_df.index.name = 'date'
    return df.join(series_df, how='inner') if not df.empty else series_df


# Loop through each series
for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_and_merge(dfs[freq], var_name, series_id, start_date, end_date)

In [8]:
dfs['M'] = dfs['M'].reset_index()
data_to_insert = dfs['M'].to_dict(orient='records')

In [9]:
collection.insert_many(data_to_insert)

In [10]:
dfs['M']

,index,date,bonds2tr,bond10tr,ppi,cpi,unrate,fedrate,m0,m3,resins,oil,indpro
0,0,1976-06-01,7.26,7.94,61.200,56.700,7.6,5.48,114400.0,1.077600e+12,33.9,12.17,43.8892
1,1,1976-07-01,7.02,7.88,61.600,57.000,7.8,5.31,115600.0,1.086300e+12,34.3,12.17,44.1328
2,2,1976-09-01,6.50,7.64,61.800,57.600,7.6,5.25,115600.0,1.110800e+12,33.9,13.90,44.5848
3,3,1976-10-01,6.24,7.49,61.900,57.900,7.7,5.02,116600.0,1.125000e+12,34.3,13.90,44.5707
4,4,1976-11-01,6.03,7.38,62.000,58.100,7.8,4.95,118300.0,1.138200e+12,34.7,13.90,45.2543
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,396,2022-07-01,2.84,2.88,272.274,294.628,3.5,1.68,5537100.0,2.170360e+13,3258.7,101.62,103.1328
397,397,2022-08-01,2.90,2.60,269.546,295.320,3.7,2.33,5582200.0,2.165990e+13,3305.9,93.67,103.2344
398,398,2022-09-01,3.51,3.26,267.898,296.539,3.5,2.56,5410900.0,2.152520e+13,3131.4,84.26,103.5283
399,399,2022-11-01,4.54,4.07,263.157,298.598,3.6,3.78,5418700.0,2.139870e+13,3126.2,84.37,103.0707
